# Initial step: load the dataset (Benchmark3)

A common dataset generated in the context of second benchmark will be used to evaluate the augmented simulators. This initial step aims at loading it once and for all.

In [1]:
import os
from pprint import pprint
from lips.neurips_benchmark import NeuripsBenchmark3
path_benchmark = os.path.join("reference_data")
neurips_benchmark3 = NeuripsBenchmark3(path_benchmark=path_benchmark,
                                       load_data_set=True)

# Training an augmented simulator 
In this benchmark the augmented simulators will predict all the power flow related variables (active $p$ and reactive $q$ powers, voltages $v$ and currents $a$) alongside the voltage angles ($\theta$) contrary to the first and second benchmarks which were concentrated only on power flow variables.

## Fully connected architecture
In this section we explain how to tune an available model. We take the example of the `FullyConnectedAS` that is an available fully connected neural network.

The first step is to create the class you want to use, with the meta parameters you want to test. For this example: 

In [2]:
path_save = os.path.join("trained_models")
if not os.path.exists(path_save):
    os.mkdir(path_save)

from tensorflow.keras.layers import Dense
from lips.augmented_simulators import FullyConnectedAS

# the three lines bellow might be familiar to the tensorflow users. They tell tensorflow to not take all
# the GPU video RAM for the model.
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
for el in physical_devices:
    tf.config.experimental.set_memory_growth(el, True)

simulator_FC = FullyConnectedAS(name="test_FullyConnectedAS_benchmark3",
                                # `attr_x` represents the variables of the dataset you want to use to predict 
                                # the output.
                                attr_x=("prod_p", "prod_v", "load_p", "load_q", "line_status", "topo_vect"),
                                # `attr_y` represents the variables of the dataset you want to predict
                                # flow variables alongside the angles are added
                                attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "load_v", "prod_q", "theta_or", "theta_ex"),
                                # `sizes_layer` represents the size of each hidden layer in the neural network. The number
                                # of layers is determined by the length of this list, for example
                                sizes_layer=(300, 300, 300, 300),
                                # `lr` is the learning rate
                                lr=3e-4, 
                                # `layer` is the type of keras layer you want to use. We don't recommend to 
                                # change it
                                layer=Dense,
                                # `layer_act` is the activation function you want to use after each layer
                                layer_act="relu",
                                # `loss` is the training loss
                                loss="mse",  # loss used to train the model
                                # `batch_size` is the size of the batch for training
                                batch_size=128)

In [ ]:
simulator_FC.train(nb_iter=200,
                   train_dataset=neurips_benchmark3.train_dataset,
                   val_dataset=neurips_benchmark3.val_dataset)

save it

In [8]:
simulator_FC.save(path_save)
simulator_FC.save_metadata(path_save)

load it

In [4]:
path_trained_models = os.path.join("trained_models")
from tensorflow.keras.layers import Dense
from lips.augmented_simulators import FullyConnectedAS

# recreate the baseline
simulator_FC = FullyConnectedAS(name="test_FullyConnectedAS_benchmark3",
                                attr_x=("prod_p", "prod_v", "load_p", "load_q", "line_status", "topo_vect"),
                                attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "load_v", "prod_q", "theta_or", "theta_ex"),
                                sizes_layer=(300, 300, 300, 300),
                                lr=3e-4, 
                                layer=Dense,
                                layer_act="relu",
                                loss="mse",  # loss used to train the model
                                batch_size=128)
# TODO create a wrapper for these 3 calls
simulator_FC.load_metadata(path_trained_models)
simulator_FC.init()
simulator_FC.restore(path_trained_models)

# Evaluate the augmented simulator 

In [5]:
fc_metrics_per_dataset = neurips_benchmark3.evaluate_augmented_simulator(simulator_FC)

A log file including some verifications is created at root directory with the name logs.log


C:\Users\Milad\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
C:\Users\Milad\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
C:\Users\Milad\Anaconda3\envs\RTE\lib\site-packages\lightsim2grid\_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")


### ML Metrics 

In [6]:
ML_metrics = 0
print("MAPE90")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["mape90"])
print("MAPE")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["mape_avg"])
print("MAE")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(fc_metrics_per_dataset["test"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.0046131610388728175,
 'a_or': 0.004655354222528099,
 'p_ex': 0.0048669815008974115,
 'p_or': 0.004879239027941079,
 'q_ex': 0.011658575615409646,
 'q_or': 0.014196010036352022,
 'theta_ex': 0.005861496950132016,
 'theta_or': nan,
 'v_ex': 0.002042967575485692,
 'v_or': 0.0020739696860987236}
MAPE
{'a_ex': 0.011037891579993783,
 'a_or': 0.011327840291289895,
 'load_v': 0.00017017968059885124,
 'p_ex': 0.011662545965693018,
 'p_or': 0.011347553230500561,
 'prod_q': 0.004511574900593358,
 'q_ex': 0.01371789179777086,
 'q_or': 0.013702970997601588,
 'theta_ex': 0.006171133177098264,
 'theta_or': nan,
 'v_ex': 0.0019030298835292323,
 'v_or': 0.0019296426453516862}
MAE
{'a_ex': 2.4643282890319824,
 'a_or': 1.6474065780639648,
 'load_v': 0.007322449702769518,
 'p_ex': 0.1342732012271881,
 'p_or': 0.13087570667266846,
 'prod_q': 0.10046952217817307,
 'q_ex': 0.0640152245759964,
 'q_or': 0.0628996342420578,
 'theta_ex': 0.04626755672660863,
 'theta_or': 0.04391781957267136,
 '

### OOD Generalization 

In [7]:
print("MAPE90")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape90"])
print("MAPE")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape_avg"])
print("MAE")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(fc_metrics_per_dataset["test_ood_topo"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.20107159106043712,
 'a_or': 0.2010885507612319,
 'p_ex': 0.23931476475031105,
 'p_or': 0.241871617038327,
 'q_ex': 0.34331784958160283,
 'q_or': 0.36442009321746427,
 'theta_ex': 0.15957073813573144,
 'theta_or': nan,
 'v_ex': 0.03118037104864476,
 'v_or': 0.030783094899299146}
MAPE
{'a_ex': 0.21175651492018047,
 'a_or': 0.2150390137056632,
 'p_ex': 0.2739599888702738,
 'p_or': 0.2756300244671787,
 'q_ex': 0.2712641773304669,
 'q_or': 0.2659483474492036,
 'theta_ex': 0.1304486416400271,
 'theta_or': nan,
 'v_ex': 0.027558933854545786,
 'v_or': 0.027649661776876232}
MAE
{'a_ex': 65.6705551147461,
 'a_or': 45.574684143066406,
 'p_ex': 3.94745135307312,
 'p_or': 4.019582748413086,
 'q_ex': 1.6291145086288452,
 'q_or': 1.5206332206726074,
 'theta_ex': 1.7184120090077706,
 'theta_or': 1.6142652562770508,
 'v_ex': 2.8946361541748047,
 'v_or': 3.7403595447540283}
NRMSE
{'a_ex': 0.056359924376010895,
 'a_or': 0.056323640048503876,
 'p_ex': 0.050281137228012085,
 'p_or': 0.050

### Physics compliance 

In [9]:
print("current pos")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(fc_metrics_per_dataset["test"][1]["BasicVerifications"]["line_status"])
print("KCL")
print("Violation percentage: ", fc_metrics_per_dataset["test"][1]["KCL"]["violation_percentage"])

current pos
0.032385
0.031465
voltage pos
0.02777
0.026965
loss
0.22728
line_status
{'a_ex_not_null': 11860.0,
 'a_or_not_null': 11860.0,
 'a_violations': 1.0,
 'p_ex_not_null': 11860.0,
 'p_or_not_null': 11860,
 'p_violations': 1.0,
 'q_ex_not_null': 11860.0,
 'q_or_not_null': 11860.0,
 'q_violations': 1.0}
KCL
Violation percentage:  93.27297297297298


In [10]:
print("current pos")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(fc_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["line_status"])
#pprint(dc_metrics_per_dataset["test_ood_topo"][1])
print("KCL")
print("Violation percentage: ", fc_metrics_per_dataset["test_ood_topo"][1]["KCL"]["violation_percentage"])

current pos
0.02754
0.02741
voltage pos
0.01226
0.012315
loss
0.362545
line_status
{'a_ex_not_null': 21843.0,
 'a_or_not_null': 21843.0,
 'a_violations': 1.0,
 'p_ex_not_null': 21843.0,
 'p_or_not_null': 21843,
 'p_violations': 1.0,
 'q_ex_not_null': 21843.0,
 'q_or_not_null': 21843.0,
 'q_violations': 1.0}
KCL
Violation percentage:  99.11331916448944


# LeapNets 

In [8]:
path_save = os.path.join("trained_models")
if not os.path.exists(path_save):
    os.mkdir(path_save)

from tensorflow.keras.layers import Dense
from lips.augmented_simulators import LeapNetAS

In [10]:
leapNet = LeapNetAS(name="test_leapNetAS_benchmark3",
                    # `attr_x` represents the variables of the dataset you want to use to predict 
                    # the output.
                    attr_x=("prod_p", "prod_v", "load_p", "load_q"),
                    # `attr_y` represents the variables of the dataset you want to predict
                    # all the powerflow variables alongside the voltage angles
                    attr_y=("a_or", "a_ex", "p_or", "p_ex", "q_ex", "q_or", "v_or", "v_ex", "theta_or", "theta_ex"),
                    # `lr` is the learning rate
                    lr=3e-4, 
                    # `layer` is the type of keras layer you want to use. We don't recommend to 
                    # change it
                    layer=Dense,
                    # `layer_act` is the activation function you want to use after each layer
                    layer_act="relu",
                    # `loss` is the training loss
                    loss="mse",  # loss used to train the model
                    # `batch_size` is the size of the batch for training
                    batch_size=128,
                    # the method used to encode the topology vector
                    topo_vect_to_tau="all")

In [11]:
leapNet.train(nb_iter=200,
              train_dataset=neurips_benchmark3.train_dataset,
              val_dataset=neurips_benchmark3.val_dataset
              )

Epoch 1/200
782/782 [==============================] - 11s 10ms/step - loss: 15.2575 - a_or_hat_loss: 1.9806 - a_ex_hat_loss: 1.9804 - p_or_hat_loss: 2.2710 - p_ex_hat_loss: 2.1339 - q_ex_hat_loss: 3.3783 - q_or_hat_loss: 2.2194 - v_or_hat_loss: 0.2185 - v_ex_hat_loss: 0.2079 - theta_or_hat_loss: 0.4250 - theta_ex_hat_loss: 0.4427 - val_loss: 0.5841 - val_a_or_hat_loss: 0.0905 - val_a_ex_hat_loss: 0.0923 - val_p_or_hat_loss: 0.0671 - val_p_ex_hat_loss: 0.0705 - val_q_ex_hat_loss: 0.1170 - val_q_or_hat_loss: 0.0767 - val_v_or_hat_loss: 0.0017 - val_v_ex_hat_loss: 0.0018 - val_theta_or_hat_loss: 0.0315 - val_theta_ex_hat_loss: 0.0350
Epoch 2/200
782/782 [==============================] - 6s 7ms/step - loss: 0.4266 - a_or_hat_loss: 0.0680 - a_ex_hat_loss: 0.0679 - p_or_hat_loss: 0.0486 - p_ex_hat_loss: 0.0509 - q_ex_hat_loss: 0.0834 - q_or_hat_loss: 0.0543 - v_or_hat_loss: 0.0013 - v_ex_hat_loss: 0.0014 - theta_or_hat_loss: 0.0243 - theta_ex_hat_loss: 0.0265 - val_loss: 0.2500 - val_a_or_

Epoch 14/200
782/782 [==============================] - 6s 8ms/step - loss: 0.0291 - a_or_hat_loss: 0.0047 - a_ex_hat_loss: 0.0050 - p_or_hat_loss: 0.0025 - p_ex_hat_loss: 0.0026 - q_ex_hat_loss: 0.0056 - q_or_hat_loss: 0.0050 - v_or_hat_loss: 6.2694e-05 - v_ex_hat_loss: 7.0395e-05 - theta_or_hat_loss: 0.0016 - theta_ex_hat_loss: 0.0019 - val_loss: 0.0333 - val_a_or_hat_loss: 0.0059 - val_a_ex_hat_loss: 0.0057 - val_p_or_hat_loss: 0.0028 - val_p_ex_hat_loss: 0.0029 - val_q_ex_hat_loss: 0.0063 - val_q_or_hat_loss: 0.0055 - val_v_or_hat_loss: 5.1563e-05 - val_v_ex_hat_loss: 5.8786e-05 - val_theta_or_hat_loss: 0.0018 - val_theta_ex_hat_loss: 0.0022
Epoch 15/200
782/782 [==============================] - 6s 8ms/step - loss: 0.0273 - a_or_hat_loss: 0.0045 - a_ex_hat_loss: 0.0048 - p_or_hat_loss: 0.0023 - p_ex_hat_loss: 0.0024 - q_ex_hat_loss: 0.0052 - q_or_hat_loss: 0.0046 - v_or_hat_loss: 5.9047e-05 - v_ex_hat_loss: 6.7455e-05 - theta_or_hat_loss: 0.0015 - theta_ex_hat_loss: 0.0018 - val_l

782/782 [==============================] - 6s 7ms/step - loss: 0.0123 - a_or_hat_loss: 0.0023 - a_ex_hat_loss: 0.0023 - p_or_hat_loss: 0.0010 - p_ex_hat_loss: 0.0010 - q_ex_hat_loss: 0.0022 - q_or_hat_loss: 0.0020 - v_or_hat_loss: 2.6928e-05 - v_ex_hat_loss: 2.9691e-05 - theta_or_hat_loss: 6.4119e-04 - theta_ex_hat_loss: 7.6925e-04 - val_loss: 0.0144 - val_a_or_hat_loss: 0.0029 - val_a_ex_hat_loss: 0.0027 - val_p_or_hat_loss: 0.0012 - val_p_ex_hat_loss: 0.0011 - val_q_ex_hat_loss: 0.0025 - val_q_or_hat_loss: 0.0024 - val_v_or_hat_loss: 1.7303e-05 - val_v_ex_hat_loss: 1.8996e-05 - val_theta_or_hat_loss: 7.4104e-04 - val_theta_ex_hat_loss: 8.6445e-04
Epoch 39/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0120 - a_or_hat_loss: 0.0023 - a_ex_hat_loss: 0.0022 - p_or_hat_loss: 9.8437e-04 - p_ex_hat_loss: 9.9172e-04 - q_ex_hat_loss: 0.0021 - q_or_hat_loss: 0.0019 - v_or_hat_loss: 2.2505e-05 - v_ex_hat_loss: 2.4207e-05 - theta_or_hat_loss: 6.2540e-04 - theta_ex_hat_loss:

Epoch 62/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0084 - a_or_hat_loss: 0.0017 - a_ex_hat_loss: 0.0016 - p_or_hat_loss: 6.8994e-04 - p_ex_hat_loss: 7.2735e-04 - q_ex_hat_loss: 0.0015 - q_or_hat_loss: 0.0013 - v_or_hat_loss: 1.6117e-05 - v_ex_hat_loss: 1.6646e-05 - theta_or_hat_loss: 4.0228e-04 - theta_ex_hat_loss: 4.9868e-04 - val_loss: 0.0099 - val_a_or_hat_loss: 0.0021 - val_a_ex_hat_loss: 0.0019 - val_p_or_hat_loss: 7.3359e-04 - val_p_ex_hat_loss: 8.4647e-04 - val_q_ex_hat_loss: 0.0018 - val_q_or_hat_loss: 0.0015 - val_v_or_hat_loss: 8.4894e-06 - val_v_ex_hat_loss: 9.9428e-06 - val_theta_or_hat_loss: 4.7504e-04 - val_theta_ex_hat_loss: 6.1340e-04
Epoch 63/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0082 - a_or_hat_loss: 0.0016 - a_ex_hat_loss: 0.0015 - p_or_hat_loss: 6.6103e-04 - p_ex_hat_loss: 6.9843e-04 - q_ex_hat_loss: 0.0015 - q_or_hat_loss: 0.0013 - v_or_hat_loss: 1.5598e-05 - v_ex_hat_loss: 1.5683e-05 - theta_or_hat_loss: 3.

Epoch 74/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0072 - a_or_hat_loss: 0.0015 - a_ex_hat_loss: 0.0014 - p_or_hat_loss: 5.7561e-04 - p_ex_hat_loss: 6.0868e-04 - q_ex_hat_loss: 0.0013 - q_or_hat_loss: 0.0011 - v_or_hat_loss: 1.2746e-05 - v_ex_hat_loss: 1.3805e-05 - theta_or_hat_loss: 3.2760e-04 - theta_ex_hat_loss: 4.2368e-04 - val_loss: 0.0093 - val_a_or_hat_loss: 0.0021 - val_a_ex_hat_loss: 0.0017 - val_p_or_hat_loss: 7.3307e-04 - val_p_ex_hat_loss: 9.6303e-04 - val_q_ex_hat_loss: 0.0015 - val_q_or_hat_loss: 0.0013 - val_v_or_hat_loss: 1.9399e-05 - val_v_ex_hat_loss: 2.0548e-05 - val_theta_or_hat_loss: 4.1021e-04 - val_theta_ex_hat_loss: 5.5679e-04
Epoch 75/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0072 - a_or_hat_loss: 0.0015 - a_ex_hat_loss: 0.0013 - p_or_hat_loss: 5.7565e-04 - p_ex_hat_loss: 5.9638e-04 - q_ex_hat_loss: 0.0013 - q_or_hat_loss: 0.0011 - v_or_hat_loss: 1.2922e-05 - v_ex_hat_loss: 1.3505e-05 - theta_or_hat_loss: 3.

...

Epoch 122/200
782/782 [==============================] - 6s 8ms/step - loss: 0.0051 - a_or_hat_loss: 0.0011 - a_ex_hat_loss: 0.0010 - p_or_hat_loss: 4.0733e-04 - p_ex_hat_loss: 4.3350e-04 - q_ex_hat_loss: 8.8541e-04 - q_or_hat_loss: 7.5242e-04 - v_or_hat_loss: 7.9289e-06 - v_ex_hat_loss: 8.5827e-06 - theta_or_hat_loss: 2.1996e-04 - theta_ex_hat_loss: 2.9323e-04 - val_loss: 0.0061 - val_a_or_hat_loss: 0.0013 - val_a_ex_hat_loss: 0.0013 - val_p_or_hat_loss: 3.9297e-04 - val_p_ex_hat_loss: 4.3785e-04 - val_q_ex_hat_loss: 0.0011 - val_q_or_hat_loss: 0.0010 - val_v_or_hat_loss: 7.5284e-06 - val_v_ex_hat_loss: 7.0780e-06 - val_theta_or_hat_loss: 2.5107e-04 - val_theta_ex_hat_loss: 3.6100e-04
Epoch 123/200
782/782 [==============================] - 6s 8ms/step - loss: 0.0049 - a_or_hat_loss: 0.0010 - a_ex_hat_loss: 9.7646e-04 - p_or_hat_loss: 3.7682e-04 - p_ex_hat_loss: 3.9485e-04 - q_ex_hat_loss: 8.3967e-04 - q_or_hat_loss: 7.3540e-04 - v_or_hat_loss: 7.0951e-06 - v_ex_hat_loss: 7.6603e-06 -

Epoch 134/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0045 - a_or_hat_loss: 9.7998e-04 - a_ex_hat_loss: 9.0785e-04 - p_or_hat_loss: 3.5368e-04 - p_ex_hat_loss: 3.7120e-04 - q_ex_hat_loss: 7.8560e-04 - q_or_hat_loss: 6.6194e-04 - v_or_hat_loss: 6.2361e-06 - v_ex_hat_loss: 6.7704e-06 - theta_or_hat_loss: 1.9391e-04 - theta_ex_hat_loss: 2.6180e-04 - val_loss: 0.0057 - val_a_or_hat_loss: 0.0013 - val_a_ex_hat_loss: 0.0011 - val_p_or_hat_loss: 4.4168e-04 - val_p_ex_hat_loss: 5.3032e-04 - val_q_ex_hat_loss: 9.9143e-04 - val_q_or_hat_loss: 7.7250e-04 - val_v_or_hat_loss: 5.4828e-06 - val_v_ex_hat_loss: 6.9120e-06 - val_theta_or_hat_loss: 2.1768e-04 - val_theta_ex_hat_loss: 3.0456e-04
Epoch 135/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0045 - a_or_hat_loss: 9.8643e-04 - a_ex_hat_loss: 9.1662e-04 - p_or_hat_loss: 3.4777e-04 - p_ex_hat_loss: 3.7146e-04 - q_ex_hat_loss: 7.8949e-04 - q_or_hat_loss: 6.7258e-04 - v_or_hat_loss: 6.7852e-06 - v_ex_ha

Epoch 146/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0044 - a_or_hat_loss: 9.4599e-04 - a_ex_hat_loss: 8.8154e-04 - p_or_hat_loss: 3.5283e-04 - p_ex_hat_loss: 3.6875e-04 - q_ex_hat_loss: 7.5397e-04 - q_or_hat_loss: 6.4054e-04 - v_or_hat_loss: 6.9616e-06 - v_ex_hat_loss: 7.3468e-06 - theta_or_hat_loss: 1.7789e-04 - theta_ex_hat_loss: 2.4947e-04 - val_loss: 0.0053 - val_a_or_hat_loss: 0.0012 - val_a_ex_hat_loss: 0.0010 - val_p_or_hat_loss: 3.7216e-04 - val_p_ex_hat_loss: 4.2256e-04 - val_q_ex_hat_loss: 9.2120e-04 - val_q_or_hat_loss: 7.5534e-04 - val_v_or_hat_loss: 5.6329e-06 - val_v_ex_hat_loss: 8.5498e-06 - val_theta_or_hat_loss: 2.2154e-04 - val_theta_ex_hat_loss: 3.0393e-04
Epoch 147/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0043 - a_or_hat_loss: 9.2575e-04 - a_ex_hat_loss: 8.7659e-04 - p_or_hat_loss: 3.2621e-04 - p_ex_hat_loss: 3.4624e-04 - q_ex_hat_loss: 7.3629e-04 - q_or_hat_loss: 6.2639e-04 - v_or_hat_loss: 7.0850e-06 - v_ex_ha

Epoch 158/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0040 - a_or_hat_loss: 8.9083e-04 - a_ex_hat_loss: 8.3705e-04 - p_or_hat_loss: 3.0782e-04 - p_ex_hat_loss: 3.2991e-04 - q_ex_hat_loss: 6.7844e-04 - q_or_hat_loss: 5.9583e-04 - v_or_hat_loss: 5.2577e-06 - v_ex_hat_loss: 5.5017e-06 - theta_or_hat_loss: 1.6375e-04 - theta_ex_hat_loss: 2.2183e-04 - val_loss: 0.0056 - val_a_or_hat_loss: 0.0012 - val_a_ex_hat_loss: 0.0011 - val_p_or_hat_loss: 3.8044e-04 - val_p_ex_hat_loss: 4.1231e-04 - val_q_ex_hat_loss: 0.0011 - val_q_or_hat_loss: 7.7984e-04 - val_v_or_hat_loss: 5.9076e-06 - val_v_ex_hat_loss: 6.3720e-06 - val_theta_or_hat_loss: 3.3329e-04 - val_theta_ex_hat_loss: 3.3106e-04
Epoch 159/200
782/782 [==============================] - 5s 7ms/step - loss: 0.0042 - a_or_hat_loss: 8.9470e-04 - a_ex_hat_loss: 8.3774e-04 - p_or_hat_loss: 3.3877e-04 - p_ex_hat_loss: 3.5009e-04 - q_ex_hat_loss: 7.1846e-04 - q_or_hat_loss: 5.9773e-04 - v_or_hat_loss: 6.8332e-06 - v_ex_hat_lo

782/782 [==============================] - 7s 9ms/step - loss: 0.0039 - a_or_hat_loss: 8.4069e-04 - a_ex_hat_loss: 7.8879e-04 - p_or_hat_loss: 3.1390e-04 - p_ex_hat_loss: 3.2385e-04 - q_ex_hat_loss: 6.6937e-04 - q_or_hat_loss: 5.7081e-04 - v_or_hat_loss: 7.7017e-06 - v_ex_hat_loss: 7.3454e-06 - theta_or_hat_loss: 1.5158e-04 - theta_ex_hat_loss: 2.0889e-04 - val_loss: 0.0058 - val_a_or_hat_loss: 0.0012 - val_a_ex_hat_loss: 0.0011 - val_p_or_hat_loss: 6.2885e-04 - val_p_ex_hat_loss: 6.2272e-04 - val_q_ex_hat_loss: 9.2818e-04 - val_q_or_hat_loss: 7.5129e-04 - val_v_or_hat_loss: 1.3041e-05 - val_v_ex_hat_loss: 1.3582e-05 - val_theta_or_hat_loss: 2.3138e-04 - val_theta_ex_hat_loss: 3.1985e-04
Epoch 181/200
782/782 [==============================] - 7s 9ms/step - loss: 0.0037 - a_or_hat_loss: 8.2698e-04 - a_ex_hat_loss: 7.7762e-04 - p_or_hat_loss: 2.9927e-04 - p_ex_hat_loss: 3.0474e-04 - q_ex_hat_loss: 6.2356e-04 - q_or_hat_loss: 5.4306e-04 - v_or_hat_loss: 5.7826e-06 - v_ex_hat_loss: 5.8107

save it

In [12]:
leapNet.save(path_save)
leapNet.save_metadata(path_save)

# Evaluate it 

In [13]:
leapNet_metrics_per_dataset = neurips_benchmark3.evaluate_augmented_simulator(leapNet)

A log file including some verifications is created at root directory with the name logs.log


### ML Metrics 

In [14]:
ML_metrics = 0
print("MAPE90")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["mape90"])
print("MAPE")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["mape_avg"])
print("MAE")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(leapNet_metrics_per_dataset["test"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.007627280422307372,
 'a_or': 0.008161371365289707,
 'p_ex': 0.0070716083679348615,
 'p_or': 0.006713402995507482,
 'q_ex': 0.01872287409967809,
 'q_or': 0.026856733574902935,
 'theta_ex': 0.008855646218386186,
 'theta_or': nan,
 'v_ex': 0.0027800687775338343,
 'v_or': 0.0027032407203839343}
MAPE
{'a_ex': 0.019798308458008926,
 'a_or': 0.020379822565709783,
 'p_ex': 0.01460759526570545,
 'p_or': 0.013359139042767925,
 'q_ex': 0.02137782497860824,
 'q_or': 0.021455024094834484,
 'theta_ex': 0.008508733215873797,
 'theta_or': nan,
 'v_ex': 0.0024370175344503144,
 'v_or': 0.0023524734635076232}
MAE
{'a_ex': 3.9130992889404297,
 'a_or': 2.7250747680664062,
 'p_ex': 0.16841043531894684,
 'p_or': 0.15682396292686462,
 'q_ex': 0.10360975563526154,
 'q_or': 0.1071694865822792,
 'theta_ex': 0.0641272084852525,
 'theta_or': 0.05289448516613774,
 'v_ex': 0.14341893792152405,
 'v_or': 0.1785714030265808}
NRMSE
{'a_ex': 0.004338228143751621,
 'a_or': 0.004624119959771633,
 'p_ex': 

### OOD Generalization

In [15]:
print("MAPE90")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape90"])
print("MAPE")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["mape_avg"])
print("MAE")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["MAE_avg"])
print("NRMSE")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][ML_metrics]["NRMSE_avg"])

MAPE90
{'a_ex': 0.13353149307680554,
 'a_or': 0.1362641806290948,
 'p_ex': 0.1498188134163031,
 'p_or': 0.13974530298697216,
 'q_ex': 0.23166653743605742,
 'q_or': 0.26613770426753536,
 'theta_ex': 0.10928676934889206,
 'theta_or': nan,
 'v_ex': 0.016116134482477444,
 'v_or': 0.015510716400258287}
MAPE
{'a_ex': 0.16660052068382186,
 'a_or': 0.16457392740735016,
 'p_ex': 0.17867681365282212,
 'p_or': 0.16137577897119917,
 'q_ex': 0.18446503347992374,
 'q_or': 0.1857152761019446,
 'theta_ex': 0.09906898033527073,
 'theta_or': nan,
 'v_ex': 0.010273143069932756,
 'v_or': 0.010370839340545229}
MAE
{'a_ex': 44.475318908691406,
 'a_or': 31.157516479492188,
 'p_ex': 2.0687756538391113,
 'p_or': 1.895237684249878,
 'q_ex': 0.9647682309150696,
 'q_or': 0.9432681202888489,
 'theta_ex': 1.3143264040202853,
 'theta_or': 1.2318205872948247,
 'v_ex': 0.5145779848098755,
 'v_or': 0.676001787185669}
NRMSE
{'a_ex': 0.04387463629245758,
 'a_or': 0.042918186634778976,
 'p_ex': 0.032695792615413666,
 'p_o

## Physics compliances

In [18]:
PhysicCompliances = 1
print("current pos")
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
#pprint(leapNet_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
#pprint(leapNet_metrics_per_dataset["test"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(leapNet_metrics_per_dataset["test"][PhysicCompliances]["BasicVerifications"]["line_status"])

current pos
0.000495
0.0003
voltage pos
loss
0.274485
line_status
{'a_ex_not_null': 0.0,
 'a_or_not_null': 0.0,
 'a_violations': 0.0,
 'p_ex_not_null': 0.0,
 'p_or_not_null': 0,
 'p_violations': 0.0,
 'q_ex_not_null': 0.0,
 'q_or_not_null': 0.0,
 'q_violations': 0.0}


In [19]:
PhysicCompliances = 1
print("current pos")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["currents"]["a_or"]["Violation_proportion"])
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["currents"]["a_ex"]["Violation_proportion"])
print("voltage pos")
#pprint(leapNet_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_or"]["Violation_proportion"])
#pprint(leapNet_metrics_per_dataset["test_ood_topo"][1]["BasicVerifications"]["voltages"]["v_ex"]["Violation_proportion"])

print("loss")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["loss"]["violation_proportion"])
print("line_status")
pprint(leapNet_metrics_per_dataset["test_ood_topo"][PhysicCompliances]["BasicVerifications"]["line_status"])
#pprint(dc_metrics_per_dataset["test_ood_topo"][1])

current pos
0.00393
0.004095
voltage pos
loss
0.39763
line_status
{'a_ex_not_null': 0.0,
 'a_or_not_null': 0.0,
 'a_violations': 0.0,
 'p_ex_not_null': 0.0,
 'p_or_not_null': 0,
 'p_violations': 0.0,
 'q_ex_not_null': 0.0,
 'q_or_not_null': 0.0,
 'q_violations': 0.0}
